# Trying R2->R examples

In [ ]:
import numpy as np
from gpcam.gp_optimizer import GPOptimizer
import matplotlib.pyplot as plt
from numpy.random import default_rng

import plotly.graph_objects as go
import numpy as np

In [ ]:
plt.figure()
plt.imshow(np.random.randn(100,100))

In [ ]:
def plot(x,y,z,data = None):
    fig = go.Figure()
    fig.add_trace(go.Surface(x = x, y = y,z=z))
    if data is not None: 
        fig.add_trace(go.Scatter3d(x=data[:,0], y=data[:,1], z=data[:,2],
                                   mode='markers'))

    fig.update_layout(title='Posterior Mean', autosize=True,
                  width=800, height=800,
                  margin=dict(l=65, r=50, b=65, t=90))

    fig.show()


In [ ]:
a = np.load(r"D:\data\us_topo.npy")
rng = default_rng()
ind = rng.choice(len(a)-1, size=50, replace=False)
points = a[ind,0:2]
values = a[ind,2:3]
print("x_min ", np.min(points[:,0])," x_max ",np.max(points[:,0]))
print("y_min ", np.min(points[:,1])," y_max ",np.max(points[:,1]))
print("length of data set: ", len(points))

index_set_bounds = np.array([[0,99],[0,248]])
hyperparameter_bounds = np.array([[0.001,1e9],[1,1000],[1,1000]])
hps_guess = np.array([4.71907062e+06, 4.07439017e+02, 3.59068120e+02])

In [ ]:
plt.figure()
plt.imshow(a[:,2].reshape(100,250))


In [ ]:
gp = GPOptimizer(2, index_set_bounds)
gp.tell(points,values)
gp.init_gp(hps_guess)
gp.train_gp(hyperparameter_bounds,pop_size = 20,tolerance = 1e-6,max_iter = 2)

In [ ]:
x_pred = np.empty((10000,2))
counter = 0
x = np.linspace(0,99,100)
y = np.linspace(0,248,100)

for i in x:
 for j in y:
   x_pred[counter] = np.array([i,j])
   counter += 1


In [ ]:
res1 = gp.posterior_mean(x_pred)

In [ ]:
res2 = gp.posterior_covariance(x_pred)
# #res3 = gp.gp.shannon_information_gain(x_pred)
# X,Y = np.meshgrid(x,y)

# PM = np.reshape(res1["f(x)"],(100,250))
# PV = np.reshape(res2["v(x)"],(100,250))
# # plot(X,Y,PM)
# # plot(X,Y,PV)


In [ ]:
X,Y = np.meshgrid(x,y)

PM = np.reshape(res1["f(x)"],(100,100))
PV = np.reshape(res2["v(x)"],(100,100))
# plot(X,Y,PM)
# plot(X,Y,PV)


In [ ]:
plot(X,Y,PM)
plot(X,Y,PV)

In [ ]:
from dask.distributed import Client, LocalCluster
cluster = LocalCluster(n_workers=10)  # Launches a scheduler and workers locally
client = Client(cluster)  #

In [ ]:
# cluster.close()

In [ ]:

next = gp.ask(position = None, n = 5, acquisition_function = "variance", bounds = None,
             method = "hgdl", pop_size = 50, max_iter = 20,
             tol = 10e-6, dask_client = client)
print(next)

In [ ]:
next['x'][:,1]

In [ ]:
fig,ax=plt.subplots(2,1,figsize=(8,4))
ax[0].imshow(PM)
ax[1].imshow(a[:,2].reshape(100,250))

# plt.scatter(next['x'][:,1]/2.48,next['x'][:,0]/.99)

In [ ]:
res2

In [ ]:
res1['x'][199]

# Trying to predict experiment point

In [ ]:
from dask.distributed import Client, LocalCluster
cluster = LocalCluster(n_workers=10)  # Launches a scheduler and workers locally
client = Client(cluster)  #

In [ ]:
import numpy as np
from gpcam.gp_optimizer import GPOptimizer
import matplotlib.pyplot as plt
from numpy.random import default_rng

import plotly.graph_objects as go
import numpy as np

In [ ]:
def init_gp(points,values,index_set_bounds,hyperparameter_bounds,hps_guess):
    gp = GPOptimizer(2, index_set_bounds)
    gp.tell(points,values)
    gp.init_gp(hps_guess, compute_device='cpu', gp_kernel_function=None,
               gp_mean_function=None, gp_kernel_function_grad=None,
               gp_mean_function_grad=None, normalize_y=False, use_inv=True,
               ram_economy=False)
    # gp.init_gp(hps_guess)
    gp.train_gp(hyperparameter_bounds,pop_size = 20,tolerance = 1e-6,max_iter = 2)
    return gp


def find_next(points,values,gp):
    gp.tell(points,values)
    gp.train_gp(hyperparameter_bounds,pop_size = 20,tolerance = 1e-6,max_iter = 2,dask_client = None)
    # gp.train_gp_async(hyperparameter_bounds, max_iter = 10000, dask_client = None)

    new = gp.ask(position = None, n = 1, acquisition_function = "shannon_ig", bounds = None,
        method = "global", pop_size = 20, max_iter = 20, tol = 10e-6, x0 = None, dask_client = None)
    
    x,y = np.round(new['x'][0]).astype(int)
    return x,y

def measure_next(a,new):
    newval = a[new[0]*250+new[1]]
    assert np.allclose(new,newval[:2]),"coordinates don't match"
    # points.append(new)
    # values.append(a[new[0]*250+new[1]])
    return newval[2]
    # new,a[new[0]*250+new[1]][2]



In [ ]:
%%time
a = np.load("/Users/davide/Downloads/us_topo.npy")
rng = default_rng()
ind = rng.choice(len(a)-1, size=5, replace=False)
points = a[ind,0:2]
values = a[ind,2:3]
print("x_min ", np.min(points[:,0])," x_max ",np.max(points[:,0]))
print("y_min ", np.min(points[:,1])," y_max ",np.max(points[:,1]))
print("length of data set: ", len(points))

index_set_bounds = np.array([[0,99],[0,249]])
hyperparameter_bounds = np.array([[0.001,1e9],[1,1000],[1,1000]])
hps_guess = np.array([4.71907062e+06, 4.07439017e+02, 3.59068120e+02])

gp = init_gp(points,values,index_set_bounds,hyperparameter_bounds,hps_guess)

In [ ]:
x_pred = np.empty((50*125,2))
counter = 0
x = np.linspace(0,99,50)
y = np.linspace(0,248,125)

for i in x:
 for j in y:
   x_pred[counter] = np.array([i,j])
   counter += 1

res1 = gp.posterior_mean(x_pred)
res2 = gp.posterior_covariance(x_pred)


X,Y = np.meshgrid(x,y)

PM = np.reshape(res1["f(x)"],(50,125))
PV = np.reshape(res2["v(x)"],(50,125))


fig,ax=plt.subplots(3,1,figsize=(8,6))
ax[0].imshow(PV,extent=[0,250,100,0])
ax[1].imshow(PM,extent=[0,250,100,0])
ax[2].imshow(a[:,2].reshape(100,250))
for aa in ax:
    aa.scatter(points[:-1,1],points[:-1,0],s=1,color='orange')
    aa.scatter(points[-1,1],points[-1,0],s=2,color='red')
    

In [ ]:
newpoint = find_next(points,values,gp)
newval = measure_next(a,newpoint)
values = np.append(values,newval)
points = np.append(points,np.array(newpoint)[None,:],axis=0)

print(values.shape,points.shape)
print(newpoint,newval)



In [ ]:
from tqdm.auto import tqdm
for i in tqdm(range(50)):
    newpoint = find_next(points,values,gp)
    newval = measure_next(a,newpoint)
    values = np.append(values,newval)
    points = np.append(points,np.array(newpoint)[None,:],axis=0)

    # print(values.shape,points.shape)
    # print(newpoint,newval)



In [ ]:
%%time
# gp.tell(points,values)
gp.train_gp_async(hyperparameter_bounds, max_iter=10000, dask_client=client, deflation_radius=None, constraints=(), local_method='L-BFGS-B', global_method='genetic')
# new = gp.ask(position = None, n = 1, acquisition_function = "variance", bounds = None, method = "global", pop_size = 20, max_iter = 20, tol = 10e-6, x0 = None, dask_client = client)



In [ ]:
x_pred = np.empty((100*250,2))
counter = 0
x = np.linspace(0,99,100)
y = np.linspace(0,249,250)

for i in x:
 for j in y:
   x_pred[counter] = np.array([i,j])
   counter += 1

res1 = gp.posterior_mean(x_pred)
res2 = gp.posterior_covariance(x_pred)


X,Y = np.meshgrid(x,y)

PM = np.reshape(res1["f(x)"],(100,250))
PV = np.reshape(res2["v(x)"],(100,250))


fig,ax=plt.subplots(3,1,figsize=(8,6))
ax[0].imshow(PV,extent=[0,250,100,0])
ax[1].imshow(PM,extent=[0,250,100,0])
ax[2].imshow(a[:,2].reshape(100,250))
for aa in ax:
    aa.scatter(points[:-1,1],points[:-1,0],s=1,color='orange')
    aa.scatter(points[-1,1],points[-1,0],s=2,color='red')
    

# Trying 1D test case

just to understand if variance depends on data

In [ ]:
import numpy as np
from gpcam.gp_optimizer import GPOptimizer
import matplotlib.pyplot as plt
from numpy.random import default_rng

import plotly.graph_objects as go
import numpy as np

In [ ]:
def ff(xx):
    return np.sin((xx - 2.5) ** 2)

In [ ]:
def init_gp(points,values,index_set_bounds,hyperparameter_bounds,hps_guess):
    gp = GPOptimizer(1, index_set_bounds)
    gp.tell(points,values)
    gp.init_gp(hps_guess, compute_device='cpu', gp_kernel_function=None,
               gp_mean_function=None, gp_kernel_function_grad=None,
               gp_mean_function_grad=None, normalize_y=False, use_inv=True,
               ram_economy=False)
    # gp.init_gp(hps_guess)
    gp.train_gp(hyperparameter_bounds,pop_size = 20,tolerance = 1e-6,max_iter = 2)
    return gp


def find_next(points,values,gp):
    gp.tell(points,values)
    gp.train_gp(hyperparameter_bounds,pop_size = 20,tolerance = 1e-6,max_iter = 2,dask_client = None)
    # gp.train_gp_async(hyperparameter_bounds, max_iter = 10000, dask_client = None)

    new = gp.ask(position = None, n = 1, acquisition_function = "shannon_ig", bounds = None,
        method = "global", pop_size = 20, max_iter = 20, tol = 10e-6, x0 = None, dask_client = None)
    
    x,y = np.round(new['x'][0]).astype(int)
    return x,y

def measure_next(a,new):
    newval = a[new[0]*250+new[1]]
    assert np.allclose(new,newval[:2]),"coordinates don't match"
    # points.append(new)
    # values.append(a[new[0]*250+new[1]])
    return newval[2]
    # new,a[new[0]*250+new[1]][2]



In [ ]:
a = np.array(np.arange(0,5,0.001))
rng = default_rng()
ind = rng.choice(len(a)-1, size=5, replace=False)

In [ ]:
a = np.array(np.arange(0,5,0.001))
a = np.c_[a,np.sin(-ff(a))]

points = a[ind,0:1]
values = a[ind,1:2]
print("x_min ", np.min(points[:])," x_max ",np.max(points[:]))
print("length of data set: ", len(points))


hps_guess = np.array([1,1,1])
hyperparameter_bounds =  np.array([[0.01,100],[0.01,100.0],[0.01,100]])
index_set_bounds = np.array([0,5])
# # hyperparameter_bounds = np.array([[0.001,1e9],[1,1000]])
# hps_guess = np.array([4.71907062e+06, 4.07439017e+02])

In [ ]:
gp = init_gp(points,values,index_set_bounds,hyperparameter_bounds,hps_guess)

In [ ]:
x_pred = np.array(np.linspace(0,5,100))

res1 = gp.posterior_mean(x_pred)
res2 = gp.posterior_covariance(x_pred)


PM0 = res1["f(x)"]
PV0 = res2["v(x)"].reshape(100)


fig,ax=plt.subplots(2,1,figsize=(8,6))
ax[0].plot(x_pred,PV0)
ax[0].plot(x_pred,PV)
ax[1].plot(x_pred,PM0)
ax[1].plot(a[:,0],a[:,1])
ax[1].plot(points,values,'.')

ax[1].fill_between(
        x_pred,
        PM0 - PV0,
        PM0 + PV0,
        alpha=0.1,
        color="black",
        label=r"$\pm$ 1 std. dev.",
    )
    

# Trying multi-output GP

using fvGPOptimizer

In [ ]:
import numpy as np
import copy as cp
from gpcam.gp_optimizer import fvGPOptimizer
import matplotlib.pyplot as plt
from numpy.random import default_rng
import time

In [ ]:
# shannon_information_gain
# def aqf_multid(x,gp):

#     ret = None
#     for i in range(gp.input_dim-1):
#         x_pred=np.c_[x,np.zeros(x.shape[0])+i].reshape(-1,gp.input_dim)
#         if ret is None:
#             ret = gp.shannon_information_gain_vec(x_pred)['sig(x)']
#         else:
#             ret += gp.shannon_information_gain_vec(x_pred)['sig(x)']
#     return ret

# upper_confidence_bounds
def aqf_multid(x, gp):
    a = 2.0  #####change here, 3.0 for 95 percent confidence interval
    norm = 1.0
    ret = None
    for i in range(gp.input_dim-1):
        x_pred=np.c_[x,np.zeros(x.shape[0])+i].reshape(-1,gp.input_dim)
        cov = gp.posterior_covariance(x_pred)["v(x)"]
        if ret is None:
            ret = cov
        else:
            ret += cov
            
    ret=a * np.sqrt(ret)

    for i in range(gp.input_dim-1):
        x_pred=np.c_[x,np.zeros(x.shape[0])+i].reshape(-1,gp.input_dim)
        mean = gp.posterior_mean(x_pred)["f(x)"]
        ret += norm * mean

    return ret

In [ ]:
def init_gp(points,values,index_set_bounds,hyperparameter_bounds,hps_guess,vp,device='cpu'):
    gp = fvGPOptimizer(input_space_dimension=2, output_space_dimension=1, output_number=2, input_space_bounds=index_set_bounds, )
    gp.tell(points,values,value_positions=vp)
    gp.init_fvgp(hps_guess,compute_device=device)
    gp.train_gp(hyperparameter_bounds,pop_size = 20,tolerance = 1e-6,max_iter = 2)
    return gp


def find_next(points,values,gp,maxnum=None,device='cpu',dask_client=None):
    if maxnum is None:
        gp.tell(points,values)
    else:
        gp.tell(points[-maxnum:],values[-maxnum:])
    gp.train_gp(
        hyperparameter_bounds,
        pop_size = 20,
        method='global',
        tolerance = 1e-6,
        max_iter = 2,
        dask_client = dask_client,
        # device=device,
    )
        # gp.train_gp_async(hyperparameter_bounds, max_iter = 10000, dask_client = None)

    new = gp.ask(
        position = None, 
        n = 1, 
        acquisition_function = aqf_multid, 
        bounds = None,
        method='global', 
        pop_size = 20, 
        max_iter = 20, 
        tol = 10e-6, 
        x0 = None, 
        dask_client = dask_client,
        # device=device,
    )
    
    x,y = np.round(new['x'][0]).astype(int)
    return x,y

def measure_next(a,new):
    newval = a[new[0]*750+new[1]]
    assert np.allclose(new,newval[:2]),"coordinates don't match"
    # points.append(new)
    # values.append(a[new[0]*250+new[1]])
    return newval[2:]
    # new,a[new[0]*250+new[1]][2]



In [ ]:
# a = np.load("/Users/davide/Downloads/us_topo.npy")
a = np.load(r"D:\data\us_topo.npy")
# a = np.c_[a,3733.5*np.sin(np.flip(a[:,2])/1000)]

In [ ]:
b=cp.deepcopy(a*np.array([1,1,0]))
b=np.append(b,(a+np.array([100,0,0]))*np.array([1,1,0]),axis=0)
b=np.append(b,(a+np.array([200,0,0]))*np.array([1,1,0]),axis=0)
b=np.append(b,(a+np.array([000,250,0]))*np.array([1,1,0]),axis=0)
b=np.append(b,(a+np.array([100,250,0]))*np.array([1,1,1]),axis=0)
b=np.append(b,(a+np.array([200,250,0]))*np.array([1,1,0]),axis=0)
b=np.append(b,(a+np.array([000,500,0]))*np.array([1,1,0]),axis=0)
b=np.append(b,(a+np.array([100,500,0]))*np.array([1,1,0]),axis=0)
b=np.append(b,(a+np.array([200,500,0]))*np.array([1,1,0]),axis=0)

b=b[np.lexsort(np.transpose(b)[::-1])]
a=b
a = np.c_[a,3733.5*np.sin(np.flip(a[:,2])/1000)]

In [ ]:
fig,ax=plt.subplots(1,2,figsize=(8,2))
ax[0].imshow(a[:,2].reshape(100*3,250*3))
ax[1].imshow(a[:,3].reshape(100*3,250*3))
# cc=(a+np.array([000,250,0]))*np.array([1,1,1])
# ax[1].scatter(a[:,0],a[:,1],c=a[:,2],marker='.',linewidth=0)


In [ ]:

rng = default_rng()
ind = rng.choice(len(a)-1, size=5, replace=False)
points = a[ind,0:2]
values = a[ind,2:4]
print("x_min ", np.min(points[:,0])," x_max ",np.max(points[:,0]))
print("y_min ", np.min(points[:,1])," y_max ",np.max(points[:,1]))
print("val_min ", np.min(values[:,1])," val_max ",np.max(values[:,1]))
print("length of data set: ", len(points))

index_set_bounds = np.array([[0,299],[0,749]])
hyperparameter_bounds = np.array([[0.001,1e9],[1,1000],[1,1000],[1,1000],[1,1000]])
hps_guess = np.array([4.71907062e+06, 4.07439017e+02, 3.59068120e+02,4e2,4e2])

z=[[[0],[1]]]
vp = np.array(z*len(ind))

gp = init_gp(points,values,index_set_bounds,hyperparameter_bounds,hps_guess,vp,device='gpu')

In [ ]:
x = np.linspace(0,299,50)
y = np.linspace(0,749,125)

# for fvGP, the third input for posteriors is the output coordinate
x_pred = np.empty((50*125,3))
counter = 0
for i in x:
 for j in y:
   x_pred[counter] = np.array([i,j,0])
   counter += 1

res10 = gp.posterior_mean(x_pred)
res20 = gp.posterior_covariance(x_pred)

x_pred = np.empty((50*125,3))
counter = 0
for i in x:
 for j in y:
   x_pred[counter] = np.array([i,j,1])
   counter += 1

res11 = gp.posterior_mean(x_pred)
res21 = gp.posterior_covariance(x_pred)


X,Y = np.meshgrid(x,y)

PM0 = np.reshape(res10["f(x)"],(50,125))
PV0 = np.reshape(res20["v(x)"],(50,125))
PM1 = np.reshape(res11["f(x)"],(50,125))
PV1 = np.reshape(res21["v(x)"],(50,125))


fig,ax=plt.subplots(2,3,figsize=(8,2))
ax[0,0].imshow(PV0,extent=[0,750,300,0])
ax[0,1].imshow(PM0,extent=[0,750,300,0])
ax[1,0].imshow(PV1,extent=[0,750,300,0])
ax[1,1].imshow(PM1,extent=[0,750,300,0])
ax[0,2].imshow(a[:,2].reshape(300,750))
ax[1,2].imshow(a[:,3].reshape(300,750))
for aa in ax.ravel():
    aa.scatter(points[:-1,1],points[:-1,0],s=1,color='orange')
    aa.scatter(points[-1,1],points[-1,0],s=2,color='red')
    

In [ ]:
plt.figure()
plt.imshow(PV0-PV1)
plt.colorbar()

In [ ]:
# x = np.linspace(0,299,50)
# y = np.linspace(0,749,125)

# x_pred = np.empty((50*125,2))
# counter = 0
# for i in x:
#  for j in y:
#    x_pred[counter] = np.array([i,j])
#    counter += 1

# res = np.reshape(aqf_multid(x_pred,gp),(50,125))



# plt.figure()
# plt.imshow(res,extent=[0,750,300,0])
# plt.colorbar()
# ax = plt.gca()
# ax.scatter(points[:-1,1],points[:-1,0],s=1,color='orange')
# ax.scatter(points[-1,1],points[-1,0],s=2,color='red')
    

In [ ]:
# gp.tell(points,values)
# gp.train_gp(hyperparameter_bounds,pop_size = 20,tolerance = 1e-6,max_iter = 2,dask_client = None)

# gp.ask(position = None, n = 1, acquisition_function = aqf_multid, bounds = None,
#     method = "global", pop_size = 20, max_iter = 20, tol = 10e-6, x0 = np.array([0,0]), dask_client = None)

# # x,y = np.round(new['x'][0]).astype(int)


In [ ]:
# import time

# t0 = time.time()

# gp.tell(points,values)
# # gp.train_gp(hyperparameter_bounds,pop_size = 20,tolerance = 1e-6,max_iter = 2,dask_client = None)
# gp.train_gp(hyperparameter_bounds, method='mcmc', pop_size=20, tolerance=1e-06, max_iter=2, constraints=(), deflation_radius=None, dask_client=None)
# # gp.train_gp_async(hyperparameter_bounds, max_iter = 10000, dask_client = None)

# # new = gp.ask(position = None, n = 1, acquisition_function = aqf_multid, bounds = None,
#     # method = "global", pop_size = 20, max_iter = 20, tol = 10e-6, x0 = None, dask_client = None)

# # x,y = np.round(new['x'][0]).astype(int)
# # newpoint = (x,y)
# t1 = time.time()

# t1-t0


In [ ]:
from tqdm.auto import tqdm
information=[]
times = []
for i in tqdm(range(500)):
    try:
        t0 = time.time()
        newpoint = find_next(points,values,gp,device='gpu')
        newval = measure_next(a,newpoint)
        values = np.append(values,np.array(newval)[None,:],axis=0)
        points = np.append(points,np.array(newpoint)[None,:],axis=0)
        times.append(time.time()-t0)
    except KeyboardInterrupt:
        break
    # ig = gp.shannon_information_gain(np.append(points[-1],np.array([0])))['prior entropy']
    # ig += gp.shannon_information_gain(np.append(points[-1],np.array([1])))['prior entropy']
    # information.append(ig)


    # print(values.shape,points.shape)
    # print(newpoint,newval)



In [ ]:
plt.figure()
plt.plot(times)

In [ ]:
plt.figure()
ax = plt.gca()

ax.plot(np.array(information[1:])-np.array(information[:-1]))
# ax.set_ylim(25,30)



In [ ]:
x_pred = np.empty((50*125,2))
counter = 0
x = np.linspace(0,299,50)
y = np.linspace(0,749,125)

for i in x:
 for j in y:
   x_pred[counter] = np.array([i,j])
   counter += 1



fig,ax=plt.subplots(3,2,figsize=(8,6))

res1 = np.reshape(aqf_multid(x_pred,gp),(50,125))
res2 = np.reshape(gp.posterior_mean(np.c_[x_pred,np.zeros(x_pred.shape[0])]+0)['f(x)'],(50,125))
ax[0,0].imshow(res1,extent=[0,750,300,0],clim=(np.quantile(res1,(0.02,0.98))))
ax[1,0].imshow(res2,extent=[0,750,300,0])
ax[2,0].imshow(a[:,2].reshape(300,750))

res1 = np.reshape(aqf_multid(x_pred,gp),(50,125))
res2 = np.reshape(gp.posterior_mean(np.c_[x_pred,np.zeros(x_pred.shape[0])+1])['f(x)'],(50,125))
ax[0,1].imshow(res1,extent=[0,750,300,0],clim=(np.quantile(res1,(0.02,0.98))))
ax[1,1].imshow(res2,extent=[0,750,300,0],clim=(np.quantile(a[:,3],(0,1))))
ax[2,1].imshow(a[:,3].reshape(300,750))

for aa in ax.flatten():
    aa.scatter(points[:-1,1],points[:-1,0],s=1,color='orange')
    aa.scatter(points[-1,1],points[-1,0],s=2,color='red')
    

In [ ]:
x_pred = np.empty((50*125,2))
counter = 0
x = np.linspace(0,99,50)
y = np.linspace(0,249,125)

for i in x:
 for j in y:
   x_pred[counter] = np.array([i,j])
   counter += 1



fig,ax=plt.subplots(3,2,figsize=(8,6))

res1 = np.reshape(aqf_multid(x_pred,gp),(50,125))
res2 = np.reshape(gp.posterior_mean(np.c_[x_pred,np.zeros(x_pred.shape[0])]+0)['f(x)'],(50,125))
ax[0,0].imshow(res1,extent=[0,250,100,0],clim=(np.quantile(res1,(0.02,0.98))))
ax[1,0].imshow(res2,extent=[0,250,100,0])
ax[2,0].imshow(a[:,2].reshape(100,250))

res1 = np.reshape(aqf_multid(x_pred,gp),(50,125))
res2 = np.reshape(gp.posterior_mean(np.c_[x_pred,np.zeros(x_pred.shape[0])+1])['f(x)'],(50,125))
ax[0,1].imshow(res1,extent=[0,250,100,0],clim=(np.quantile(res1,(0.02,0.98))))
ax[1,1].imshow(res2,extent=[0,250,100,0],clim=(np.quantile(a[:,3],(0,1))))
ax[2,1].imshow(a[:,3].reshape(100,250))

for aa in ax.flatten():
    aa.scatter(points[:-1,1],points[:-1,0],s=1,color='orange')
    aa.scatter(points[-1,1],points[-1,0],s=2,color='red')
    

In [ ]:

gp = fvGPOptimizer(input_space_dimension=2, output_space_dimension=1, output_number=2, input_space_bounds=index_set_bounds)
vp = np.array([[[0.],[1.]],[[0.],[1.]],[[0.],[1.]],[[0.],[1.]],[[0.],[1.]]])
gp.tell(points,values,value_positions=vp)
gp.init_fvgp(hps_guess)
# gp.train_gp(hyperparameter_bounds,pop_size = 20,tolerance = 1e-6,max_iter = 2)



# Basic GPCam example

In [ ]:
from gpcam.autonomous_experimenter import AutonomousExperimenterGP
import numpy as np

def instrument(data):
    for entry in data:
        entry["value"] = np.sin(np.linalg.norm(entry["position"]))
    return data

##set up your parameter space
parameters = np.array([[3.0,45.8],
                       [4.0,47.0]])

##set up some hyperparameters, if you have no idea, set them to 1 and make the training bounds large
init_hyperparameters = np.array([1,1,1])
hyperparameter_bounds =  np.array([[0.01,100],[0.01,100.0],[0.01,100]])

##let's initialize the autonomous experimenter ...
my_ae = AutonomousExperimenterGP(parameters, init_hyperparameters,
                                 hyperparameter_bounds,instrument_func = instrument,  
                                 init_dataset_size=10)
#...train...
my_ae.train()

#...and run. That's it. You successfully executed an autonomous experiment.
my_ae.go(N = 100)